In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score,recall_score,precision_recall_curve
from matplotlib import pyplot as plt
import string
import numpy as np
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')

ImportError: cannot import name 'plot_precision_recall_curve' from 'sklearn.metrics' (c:\Users\sergi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\__init__.py)

In [3]:
df = pd.read_csv("data/toxic.csv",sep=",")

In [4]:
df.shape

(14412, 2)

In [5]:
df.head()

comment  toxic
0               Верблюдов-то за что? Дебилы, бл...\n    1.0
1  Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                          Собаке - собачья смерть\n    1.0
3  Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4  тебя не убедил 6-страничный пдф в том, что Скр...    1.0

In [6]:
df["toxic"] = df["toxic"].apply(int)

In [7]:
df.head(5)

comment  toxic
0               Верблюдов-то за что? Дебилы, бл...\n      1
1  Хохлы, это отдушина затюканого россиянина, мол...      1
2                          Собаке - собачья смерть\n      1
3  Страницу обнови, дебил. Это тоже не оскорблени...      1
4  тебя не убедил 6-страничный пдф в том, что Скр...      1

In [8]:
df["toxic"].value_counts()

toxic
0    9586
1    4826
Name: count, dtype: int64

In [9]:
for i in df[df["toxic"] == 1]["comment"].head(5):
    print (i)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



In [10]:
for i in df[df["toxic"] == 0]["comment"].head(5):
    print (i)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

In [11]:
train_df , test_df = train_test_split(df,test_size=500)

In [12]:
test_df.shape

(500, 2)

In [13]:
test_df["toxic"].value_counts()

toxic
0    322
1    178
Name: count, dtype: int64

In [14]:
train_df["toxic"].value_counts()

toxic
0    9264
1    4648
Name: count, dtype: int64

In [15]:
sentence_exemple = df.iloc[1]['comment']
tokens = word_tokenize(sentence_exemple,language="russian")
tokens_without_punctuatuion = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuatuion = [i for i in tokens_without_punctuatuion if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuatuion]

In [16]:
print(f"Исходный текст: {sentence_exemple}")
print("------------------------------------")
print(f"Токены: {tokens}")
print("------------------------------------")
print(f"Токены без пунктуации: {tokens_without_punctuatuion}")
print("------------------------------------")
print(f"Токены без пунктуации и без стоп слов: {tokens_without_stop_words_and_punctuatuion}")
print("------------------------------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("------------------------------------")

Исходный текст: Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

------------------------------------
Токены: ['Хохлы', ',', 'это', 'отдушина', 'затюканого', 'россиянина', ',', 'мол', ',', 'вон', ',', 'а', 'у', 'хохлов', 'еще', 'хуже', '.', 'Если', 'бы', 'хохлов', 'не', 'было', ',', 'кисель', 'их', 'бы', 'придумал', '.']
------------------------------------
Токены без пунктуации: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'а', 'у', 'хохлов', 'еще', 'хуже', 'Если', 'бы', 'хохлов', 'не', 'было', 'кисель', 'их', 'бы', 'придумал']
------------------------------------
Токены без пунктуации и без стоп слов: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'хохлов', 'хуже', 'Если', 'хохлов', 'кисель', 'придумал']
------------------------------------
Токены после стемминга: ['хохл', 'эт', 'отдушин', 'затюкан', 'россиянин', 'мол', 'вон', 'хохл', 'хуж', 'есл', 'хохл', 'кисел

In [17]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence:str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence,language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens
    

In [18]:
tokenize_sentence(sentence_exemple)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [19]:
vectorizer = TfidfVectorizer(tokenizer= lambda x: tokenize_sentence(x,remove_stop_words=True))

In [20]:
features = vectorizer.fit_transform(train_df["comment"])

c:\Users\sergi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df["toxic"])

LogisticRegression(random_state=0)

In [22]:
model.predict(features[0])

array([1], dtype=int64)

In [23]:
train_df["comment"].iloc[0]

'согласна, типичный спермобак'

In [24]:
model_pipline = Pipeline([
    ("vectorizer",TfidfVectorizer(tokenizer=lambda x : tokenize_sentence(x,remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0))
])

In [25]:
model_pipline.fit(train_df["comment"],train_df["toxic"])

c:\Users\sergi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x00000269DB01D620>)),
                ('model', LogisticRegression(random_state=0))])

In [26]:
model_pipline.predict(["Привет, чем отличается матан от бутана? Формулами "])

array([0], dtype=int64)

In [27]:
model_pipline.predict(["Иди отсюда маленький Джон"])

array([1], dtype=int64)

In [28]:
precision_score(y_true=test_df["toxic"], y_pred=model_pipline.predict(test_df["comment"]))

0.9029850746268657

In [29]:
recall_score(y_true=test_df["toxic"],y_pred=model_pipline.predict(test_df["comment"]))

0.6797752808988764